In [1]:
"""
Name: movement.ipynb
Authors: Stephan Meighen-Berger, Martina Karl
Example how to create a video of the simulation
Requires mpl_toolkits and imageio
"""

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import matplotlib.colors as colors
import imageio

In [3]:
# Adding path to module
sys.path.append("../")

In [4]:
# picture path
PICS = '../pics/'

In [5]:
# Module imports
from fourth_day import Fourth_Day as FD
from fourth_day import config

In [6]:
# Some example settings
# Scenario
config['scenario']['population size'] = 1
config['scenario']['duration'] = 500
config['scenario']['organism movement'] = False
config['scenario']['exclusion'] = True
config['scenario']['injection']['rate'] = 1e-2
config['scenario']['injection']['y range'] = [5., 15.]
config['scenario']['light prop'] = {
            "switch": True,
            "x_pos": 5.,
            "y_pos": 10.,
        }
config['scenario']['detector'] = {
    "response": True,
    "type": "Flat",
    "mean detection prob": 0.5
}
# Organisms 
config['organisms']['emission fraction'] = 0.1
config['organisms']['alpha'] = 2.
config['organisms']['photon yield'] = 1e6
# Geometry
config['geometry']['volume'] = {
    'function': 'rectangle',
    'x_length': 40.,
    'y_length': 20.,
    'offset': None,
}
config['geometry']['observation'] = {
    'function': 'rectangle',
    'x_length': 40.,
    'y_length': 12.,
    "offset": np.array([0., 4.]),
}
config['geometry']["exclusion"] = {
    "function": "sphere",
    "radius": 0.3,
    "x_pos": 5.,
    "y_pos": 10.,
}
# Water
config['water']['model']['name'] = 'custom' # 'potential cylinder'
config['water']['model']['directory'] = "../data/current/cyclic/"
config['water']['model']['time step'] = 50
config['advanced']['starting step'] = 575

In [7]:
statistics = []
wavelengths = []
measured = []
seeds = [10, 1337]
for seed in seeds:
    # General
    config["general"]["random state seed"] = seed
    # Creating a fourth_day object
    fd = FD()
    # Launching solver
    fd.sim()
    statistics.append(fd.statistics)
    wavelengths.append(fd.wavelengths)
    measured.append(fd.measured)

In [8]:
# Plotting standards
std_size = 12
fontsize = 12.
lw=3.
h_length=1.5

In [9]:
# Random noise
noise = []
for i, _ in enumerate(fd.wavelengths):
    noise.append(np.abs(np.random.normal(2e2, 1e2, len(fd.measured[:, 0]))))
noise = np.array(noise)
wavelength_edges = np.append(fd.wavelengths, fd.wavelengths[-1]+10).astype(int)
wavelength_widths = np.diff(wavelength_edges)
wavelength_centers = (wavelength_edges[:-1] + wavelength_edges[1:]) / 2
for i, _ in tqdm(enumerate(fd.t)):
    figure, ((ax, ax3), (ax1, ax4), (ax2, ax5)) = plt.subplots(3, 2, figsize=(std_size, std_size * 6. / 8.))
    # The emitters
    emission_mask = statistics[0][i].loc[:, 'is_emitting']
    ax.scatter(statistics[0][i].loc[~emission_mask, 'pos_x'].values,
               statistics[0][i].loc[~emission_mask, 'pos_y'].values,
               color='b',
               s=30.
    )
    ax.scatter(statistics[0][i].loc[emission_mask, 'pos_x'].values,
               statistics[0][i].loc[emission_mask, 'pos_y'].values,
               color='r',
               s=30.
    )
    # Exclusion
    # theta goes from 0 to 2pi
    theta = np.linspace(0, 2*np.pi, 50)
    # the radius of the circle
    r = config['geometry']['exclusion']['radius']
    # compute x1 and x2
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    ax.plot(x1 + config['geometry']['exclusion']['x_pos'], x2 + config['geometry']['exclusion']['y_pos'],
            lw=lw, color='k')
    # Additional options
    ax.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax.title=ax.set_title('Population t = %.f s' %(fd.t[i] / 5.), fontsize=fontsize)
    ax.set_xlim(0., 40.)
    ax.set_ylim(0., 20.)
    # -----------------------------------------------------------------------------------------------------
    # The light emission curve
    for j, _ in enumerate(fd.wavelengths):
        ax1.plot((fd.t / 5.)[:i+1],
                 measured[0][:i+1, j] + noise[j, :i+1],
                 lw=lw, label='%.d' % fd.wavelengths[j])
    ax1.plot((fd.t / 5.)[:i+1],
             np.sum(measured[0][:i+1, :], axis=1) + np.sum(noise[:, :i+1], axis=0),
             lw=lw, label='Total', color='k')
    #ax1.set_ylim(1e6, 1e11)
    ax1.set_xscale('linear')
    ax1.set_yscale('log')
    ax1.set_xlabel(r'$t\;[$s$]$', fontsize=fontsize)
    ax1.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax1.grid(True)
    h, l = ax1.get_legend_handles_labels()
    lgd1 = ax1.legend(h,l, loc=9, bbox_to_anchor=(0.9, +1.),
                      ncol=1, fontsize=fontsize, handlelength=h_length,
                      fancybox=True, frameon=True)
    ax1.set_xlim(0., max(fd.t / 5.))
    ax1.set_ylim(2e1,1e6)
    # -----------------------------------------------------------------------------------------------------
    # Spectrum change
    ax2.bar(fd.wavelengths.astype(int), measured[0][i, :] + noise[:, i], width=wavelength_widths, align='edge')
    ax2.set_xscale('linear')
    ax2.set_yscale('linear')
    ax2.set_xlabel(r'$\lambda\;[$nm$]$', fontsize=fontsize)
    ax2.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax2.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax2.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax2.set_xlim(min(wavelength_edges), max(wavelength_edges))
    ax2.set_ylim(0.,5e4)
    ax2.grid(True)
    # ----------------------------------------------------------------------------------------------------
    emission_mask = statistics[0][i].loc[:, 'is_emitting']
    ax3.scatter(statistics[0][i].loc[~emission_mask, 'pos_x'].values,
               statistics[0][i].loc[~emission_mask, 'pos_y'].values,
               color='b',
               s=30.
    )
    ax3.scatter(statistics[0][i].loc[emission_mask, 'pos_x'].values,
               statistics[0][i].loc[emission_mask, 'pos_y'].values,
               color='r',
               s=30.
    )
    # Exclusion
    # theta goes from 0 to 2pi
    theta = np.linspace(0, 2*np.pi, 50)
    # the radius of the circle
    r = config['geometry']['exclusion']['radius']
    # compute x1 and x2
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    ax3.plot(x1 + config['geometry']['exclusion']['x_pos'], x2 + config['geometry']['exclusion']['y_pos'],
            lw=lw, color='k')
    # Additional options
    ax3.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax3.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax3.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax3.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax3.title=ax.set_title('Population t = %.f s' %(fd.t[i] / 5.), fontsize=fontsize)
    ax3.set_xlim(0., 40.)
    ax3.set_ylim(0., 20.)
    # -----------------------------------------------------------------------------------------------------
    # The light emission curve
    for j, _ in enumerate(fd.wavelengths):
        ax4.plot((fd.t / 5.)[:i+1],
                 measured[1][:i+1, j] + noise[j, :i+1],
                 lw=lw, label='%.d' % fd.wavelengths[j])
    ax4.plot((fd.t / 5.)[:i+1],
             np.sum(measured[1][:i+1, :], axis=1) + np.sum(noise[:, :i+1], axis=0),
             lw=lw, label='Total', color='k')
    #ax1.set_ylim(1e6, 1e11)
    ax4.set_xscale('linear')
    ax4.set_yscale('log')
    ax4.set_xlabel(r'$t\;[$s$]$', fontsize=fontsize)
    ax4.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax4.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax4.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax4.grid(True)
    h, l = ax4.get_legend_handles_labels()
    lgd1 = ax4.legend(h,l, loc=9, bbox_to_anchor=(0.9, +1.),
                      ncol=1, fontsize=fontsize, handlelength=h_length,
                      fancybox=True, frameon=True)
    ax4.set_xlim(0., max(fd.t / 5.))
    ax4.set_ylim(2e1,1e6)
    # -----------------------------------------------------------------------------------------------------
    # Spectrum change
    ax5.bar(fd.wavelengths.astype(int), measured[1][i, :] + noise[:, i], width=wavelength_widths, align='edge')
    ax5.set_xscale('linear')
    ax5.set_yscale('linear')
    ax5.set_xlabel(r'$\lambda\;[$nm$]$', fontsize=fontsize)
    ax5.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax5.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax5.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax5.set_xlim(min(wavelength_edges), max(wavelength_edges))
    ax5.set_ylim(0.,5e4)
    ax5.grid(True)
    # ----------------------------------------------------------------------------------------------------
    figure.savefig(PICS + '\\Frames\\frame_%d.jpeg' %i, quality=10)
    plt.close(figure)

500it [05:36,  1.48it/s]


In [10]:
images = []
filenames = np.array([
    PICS + 'Frames\\frame_%d.jpeg' %i
    for i in range(len(fd.t))
])
for filename in filenames[::3]:
    images.append(imageio.imread(filename))
imageio.mimsave(PICS + '%s.gif' %config['water']['model']['name'],
                images, fps=10)